working version of subtagging
to be done:
- filtering json responses only for selected files
- words cloud for each tag subset


In [68]:
import pandas as pd
import os
TEXTS_PATH = os.path.join(os.path.dirname(os.getcwd()), "src", "text_extraction", "txt_selection")
RESPONSES_PATH = os.path.join(os.path.dirname(os.getcwd()), "src", "scraping", "output", "responses_2018_now.json")

pd.set_option('display.max_rows', 10)

In [70]:
TEXTS_PATH
txt_filenames = [f for f in os.listdir(TEXTS_PATH) if f.endswith(".txt")]
txt_filenames

['111-111-2018-001734_FR.txt',
 '111-111-2018-001734_NL.txt',
 '111-111-2018-001735_FR.txt',
 '111-111-2018-001735_NL.txt',
 '111-111-2018-002986_FR.txt',
 '111-111-2018-002986_NL.txt',
 '111-111-2018-008643_FR.txt',
 '111-111-2018-008643_NL.txt',
 '111-111-2018-008644_FR.txt',
 '111-111-2018-008644_NL.txt',
 '111-111-2018-009675_FR.txt',
 '111-111-2018-009675_NL.txt',
 '111-111-2018-009676_FR.txt',
 '111-111-2018-009676_NL.txt',
 '111-111-2018-009677_FR.txt',
 '111-111-2018-009677_NL.txt',
 '111-111-2018-011427_FR.txt',
 '111-111-2018-011427_NL.txt',
 '111-111-2018-012196_FR.txt',
 '111-111-2018-012196_NL.txt',
 '111-111-2018-012197_FR.txt',
 '111-111-2018-012197_NL.txt',
 '111-111-2018-013525_FR.txt',
 '111-111-2018-013525_NL.txt',
 '111-111-2019-001595_FR.txt',
 '111-111-2019-001595_NL.txt',
 '111-111-2019-001596_FR.txt',
 '111-111-2019-001596_NL.txt',
 '111-111-2019-008438_FR.txt',
 '111-111-2019-008438_NL.txt',
 '111-111-2019-008439_FR.txt',
 '111-111-2019-008439_NL.txt',
 '111-11

In [3]:
RESPONSES_PATH

'F:\\belearner\\KPMG-Usecase\\src\\scraping\\output\\responses_2018_now.json'

In [41]:
responses = pd.read_json(RESPONSES_PATH)
responses.columns
#https://raw.githubusercontent.com/naomithiru/KPMG-Usecase/philippe/src/scraping/output/responses_2018_now.json

Index(['jcId', 'jcFr', 'jcNl', 'titleFr', 'titleNl', 'themesFr', 'themesNl',
       'signatureDate', 'validityDate', 'depositDate', 'recordDate',
       'depositRegistrationDate', 'depositNumber', 'enforced',
       'royalDecreeDate', 'noticeDepositMBDate', 'publicationRoyalDecreeDate',
       'correctedDate', 'documentLink', 'documentSize', 'scopeFr', 'scopeNl',
       'noScopeFr', 'noScopeNl'],
      dtype='object')

In [47]:
responses.head(1)

,jcId,jcFr,jcNl,titleFr,titleNl,themesFr,themesNl,signatureDate,validityDate,depositDate,...,royalDecreeDate,noticeDepositMBDate,publicationRoyalDecreeDate,correctedDate,documentLink,documentSize,scopeFr,scopeNl,noScopeFr,noScopeNl
0,1020200,SOUS-COMMISSION PARITAIRE DE L'INDUSTRIE DES C...,PARITAIR SUBCOMITE VOOR HET BEDRIJF DER HARDST...,conditions de rémunération,loonvoorwaarden,"[SALAIRES, ECOCHÈQUES, TRAVAIL EN ÉQUIPE ET DE...","[LONEN, ECOCHEQUES, PLOEGEN- EN NACHTARBEID, E...",2018-10-23T10:00:00.000+0000,2019-12-31T11:00:00.000+0000,2018-10-31T11:00:00.000+0000,...,2019-02-05T11:00:00.000+0000,2018-12-18T11:00:00.000+0000,2019-02-20T11:00:00.000+0000,None,10202/10202-2018-012766.pdf,1 Mb,None,None,None,None


In [63]:
json file to be filtered

,jcId,jcFr,titleFr,themesFr,scopeFr,noScopeFr,documentLink
0,1020200,SOUS-COMMISSION PARITAIRE DE L'INDUSTRIE DES C...,conditions de rémunération,"[SALAIRES, ECOCHÈQUES, TRAVAIL EN ÉQUIPE ET DE...",None,None,10202/10202-2018-012766.pdf
1,1020500,SOUS-COMMISSION PARITAIRE DE L'INDUSTRIE DES C...,conditions de travail et de rémunération,"[SALAIRES, TRAVAIL EN ÉQUIPE ET DE NUIT, PRIME...",None,None,10205/10205-2018-004963.pdf


In [ ]:
columns = ['jcId', 'jcFr', 'titleFr', 'themesFr', 'scopeFr', 'noScopeFr', 'documentLink']
responses2 = responses.copy(deep=True)[:10].loc[:,columns]
responses2 = responses2.loc[:, columns]
responses2.head(2)

In [ ]:
themes = []
for r in  responses2.themesFr:
    for t in r:
        if t not in themes:
            themes += [t]
themes

In [39]:
#for t in themes:
t = themes[0]
mask = responses2.themesFr.apply(lambda x: t in x)
r_t = responses2[mask]
print(t)
print(r_t)

SALAIRES
      jcId                                               jcFr  \
0  1020200  SOUS-COMMISSION PARITAIRE DE L'INDUSTRIE DES C...   
1  1020500  SOUS-COMMISSION PARITAIRE DE L'INDUSTRIE DES C...   

                                                jcNl  \
0  PARITAIR SUBCOMITE VOOR HET BEDRIJF DER HARDST...   
1  PARITAIR SUBCOMITE VOOR HET BEDRIJF DER PORSEL...   

                                    titleFr                      titleNl  \
0                conditions de rémunération              loonvoorwaarden   
1  conditions de travail et de rémunération  loon- en arbeidsvoorwaarden   

                                            themesFr  \
0  [SALAIRES, ECOCHÈQUES, TRAVAIL EN ÉQUIPE ET DE...   
1  [SALAIRES, TRAVAIL EN ÉQUIPE ET DE NUIT, PRIME...   

                                            themesNl  \
0  [LONEN, ECOCHEQUES, PLOEGEN- EN NACHTARBEID, E...   
1  [LONEN, PLOEGEN- EN NACHTARBEID, EINDEJAARSPRE...   

                  signatureDate                  validityDa

In [65]:
responses2["txt_name"] = responses2["documentLink"]
responses2["txt_name"] = responses2["txt_name"].str.replace("/","-")
responses2["documentLink"] = responses2["documentLink"].str.replace(".pdf","_FR.txt")
responses2.txt_name

t:\users\fra\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


0    10202-10202-2018-012766_FR.txt
1    10205-10205-2018-004963_FR.txt
2    10205-10205-2018-001876_FR.txt
3    10205-10205-2018-001877_FR.txt
4    10205-10205-2018-001878_FR.txt
5    10205-10205-2018-001879_FR.txt
6    10205-10205-2018-001881_FR.txt
7    10206-10206-2018-009345_FR.txt
8    10206-10206-2018-008502_FR.txt
9    10206-10206-2018-006587_FR.txt
Name: txt_name, dtype: object

In [69]:
for filename in responses2.txt_name:
    filename = '111-111-2018-001734_FR.txt'
    file_path = os.path.join(TEXTS_PATH, filename)
    with open(file_path, 'r') as file:
        responses2.loc[responses2.txt_name==filename,"txt_body"] = file.read().rstrip('\n')

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 516: character maps to <undefined>